In [0]:
%run "../../config/project_config"

In [0]:
# src/01_bronze/ingest_raw_data

from pyspark.sql.functions import current_timestamp, col

# 1. Şemayı (Database) garantiye alalım
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {db_bronze}")

def ingest_bronze_layer_volume():
    print(f"🚀 Bronze Layer (Unity Catalog) Başlıyor... Hedef: {db_bronze}\n")
    
    for table_name, filename in datasets.items():
        # Volume üzerindeki tam dosya yolu
        full_path = f"{raw_data_path}{filename}"
        
        try:
            # A. Oku (CSV)
            # Not: Unity Catalog'da _metadata sütunu otomatik olarak erişilebilir durumdadır.
            df = spark.read.format("csv") \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .load(full_path)
            
            # B. Zenginleştir (MODERN YÖNTEM)
            # input_file_name() yerine _metadata.file_path kullanıyoruz.
            df_enriched = df \
                .withColumn("ingestion_date", current_timestamp()) \
                .withColumn("source_file", col("_metadata.file_path"))
            
            # C. Yaz (Managed Delta Table)
            target_table = f"{db_bronze}.{table_name}"
            
            df_enriched.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(target_table)
            
            print(f"✅ Tablo Oluştu: {target_table}")
            
        except Exception as e:
            print(f"❌ HATA ({table_name}): {e}")

    print("\n🎉 TEBRİKLER! Bronze Katmanı Tamamlandı.")

# Fonksiyonu çalıştır
ingest_bronze_layer_volume()